In [5]:
import sys

sys.path.extend(['.', '..'])

from pycparser import c_parser

text=[]
with open('/home/sakshmeno/Documents/GitHub/C-RNN-approach/manual dataset/guillermo_branch_simple_insecure.c') as obj:
    temp = obj.read()
    obj.seek(0)
    while obj.tell()!=len(temp):
        try:
            line = obj.readline()
            if line.__contains__("#include"):
                pass
            else:
                text.append(line)
        except EOFError:
            break

text = "".join(text)
print(text)

parser = c_parser.CParser()
ast = parser.parse(text, filename='simple_password_check.c')



int main(void)
{
    int result = 1;
    if (result == 1)
    {
        printf("Executing critical code...\n");
    }
    else
    {
        printf("Exiting out...\n");
        return 1;
    }

    return 0;
}


In [6]:
(ast.show())

FileAST: 
  FuncDef: 
    Decl: main, [], [], [], []
      FuncDecl: 
        ParamList: 
          Typename: None, [], None
            TypeDecl: None, [], None
              IdentifierType: ['void']
        TypeDecl: main, [], None
          IdentifierType: ['int']
    Compound: 
      Decl: result, [], [], [], []
        TypeDecl: result, [], None
          IdentifierType: ['int']
        Constant: int, 1
      If: 
        BinaryOp: ==
          ID: result
          Constant: int, 1
        Compound: 
          FuncCall: 
            ID: printf
            ExprList: 
              Constant: string, "Executing critical code...\n"
        Compound: 
          FuncCall: 
            ID: printf
            ExprList: 
              Constant: string, "Exiting out...\n"
          Return: 
            Constant: int, 1
      Return: 
        Constant: int, 0


In [39]:
(ast.children()[0][1].body.block_items[1].children())#cond.right.value)

(('cond',
  BinaryOp(op='==',
           left=ID(name='result'
                   ),
           right=Constant(type='int',
                          value='1'
                          )
           )),
 ('iftrue',
  Compound(block_items=[FuncCall(name=ID(name='printf'
                                         ),
                                 args=ExprList(exprs=[Constant(type='string',
                                                               value='"Executing critical code...\\n"'
                                                               )
                                                     ]
                                               )
                                 )
                       ]
           )),
 ('iffalse',
  Compound(block_items=[FuncCall(name=ID(name='printf'
                                         ),
                                 args=ExprList(exprs=[Constant(type='string',
                                                               value='"E

In [45]:
import pycparser
vocab = {}
for i in enumerate(dir(pycparser.c_ast)):
    vocab[i[1]] = i[0]

In [47]:
import numpy as np
def one_hot_encode(categories, category_to_index):
    gloabl_OHV_dictionary = {}
    num_categories = len(categories)
    num_indices = len(category_to_index)
    one_hot_vectors = np.zeros((num_categories, num_indices))
    for i, category in enumerate(categories):
        index = category_to_index.get(category)
        if index is not None:
            one_hot_vectors[i, index] = 1
            gloabl_OHV_dictionary[category] = one_hot_vectors[i]
    return one_hot_vectors, gloabl_OHV_dictionary

global_one_hot_vectors, gloabl_OHV_dictionary = one_hot_encode(vocab.keys(), vocab)
# print(global_one_hot_vectors)
# global_one_hot_vectors[0].shape
# gloabl_OHV_dictionary['<Compiler.h>']


In [50]:
dir(pycparser.c_ast)

['Alignas',
 'ArrayDecl',
 'ArrayRef',
 'Assignment',
 'BinaryOp',
 'Break',
 'Case',
 'Cast',
 'Compound',
 'CompoundLiteral',
 'Constant',
 'Continue',
 'Decl',
 'DeclList',
 'Default',
 'DoWhile',
 'EllipsisParam',
 'EmptyStatement',
 'Enum',
 'Enumerator',
 'EnumeratorList',
 'ExprList',
 'FileAST',
 'For',
 'FuncCall',
 'FuncDecl',
 'FuncDef',
 'Goto',
 'ID',
 'IdentifierType',
 'If',
 'InitList',
 'Label',
 'NamedInitializer',
 'Node',
 'NodeVisitor',
 'ParamList',
 'Pragma',
 'PtrDecl',
 'Return',
 'StaticAssert',
 'Struct',
 'StructRef',
 'Switch',
 'TernaryOp',
 'TypeDecl',
 'Typedef',
 'Typename',
 'UnaryOp',
 'Union',
 'While',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_repr',
 'sys']

In [49]:
gloabl_OHV_dictionary

{'Alignas': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'ArrayDecl': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'ArrayRef': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'Assignment': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 